# colab版VMDサイジング

## 機能概要

vmd(MMDモーションデータ)を、指定されたモデルに適用したサイズで再生成します。

ねんどろ風など、頭身が大幅に異なり、腕と頭の接触を避けたい場合、モデルの頂点情報を指定すると、接触回避処理を追加で行います。

## colab (Google Colaboratory)とは


> Google Colaboratory(略称：colab (コラボ)) は、完全にクラウドで実行される Jupyter ノートブック環境です。設定不要で、無料でご利用になれます。

> Jupyter Notebook (ジュピター・ノートブック) とは、ノートブックと呼ばれる形式で作成したプログラムを実行し、実行結果を記録しながら、データの分析作業を進めるためのツールです。

Chrome もしくは Firefoxがあれば、後は全部Googleのクラウド上で、VMDサイジングを実行することができます。
実行結果のVMD等は、ご自分のローカル環境にダウンロードする事ができます。

使い方に少しクセがあるので、最初は[MMD自動トレースの導入編](https://colab.research.google.com/github/miu200521358/motion_trace_colab/blob/master/AutoTraceIntroduction.ipynb)から始めてください。慣れたら本ページの実行のみで大丈夫です。

## 実行手順

このノートブックでは、VMDサイジングの準備と実行を行います。

ニコニコ動画で、実行手順の動画を公開しています。参考にしてください。

[VMDサイジングツールを作ってみた【ver1.00】
](https://www.nicovideo.jp/watch/sm35044496)


画面左上の「＞」をクリックして下さい。目次が開きます。

![目次](https://drive.google.com/uc?export=view&id=1x8AdFNmsIQPrtYptBf_NXPRNBJF8ON8z)


ノートブックを上から順に確認し、以下手順をひとつずつ実行してください。

- **「Googleドライブとの連携」**
  - Googleドライブとの連携ができたことを確認します
  - 連携のやり方は、[MMD自動トレースの導入編](https://colab.research.google.com/github/miu200521358/motion_trace_colab/blob/master/AutoTraceIntroduction.ipynb)をご確認ください
- **「VMDサイジング準備」**
    - 準備セクションのセルを実行します
      - この処理で、MMDサイジングに必要なプログラムやデータがすべてcolab上に作成されます。
      - 大体1分くらいかかります。
- **「VMDサイジング実行」**
  - 実行セクションのセルを上から順番に1つずつ実行します
    - パラメーターの設定
    - 移動系ボーン（センター・足IK等）の縮尺変換処理を行います。
    - 頂点CSVがある場合、頭部と腕の接触回避処理が追加で行います。
    - 6000Fで大体2分くらいかかります。


# VMDサイジング準備

## Googleドライブとの連携

Googleドライブの「`vmdsizing`」 フォルダと連携します。

Googleドライブに「`vmdsizing`」 フォルダを作成し、下のセルを実行してください。

In [0]:
from google.colab import drive
import os

# Googleドライブマウント
drive.mount('/gdrive')

# 起点ディレクトリ
base_path = "/gdrive/My Drive/vmdsizing"

! echo "vmdsizingフォルダの中身 -----------"
! ls -l "$base_path"
! echo "--------------------"

## 準備実行

下のセルを実行して下さい。

In [0]:
# 処理時間計測のための開始時間
import time
start_time = time.time()

# vmd_sizing のライブラリ

! sudo apt-get install python3-pyqt5  
! sudo apt-get install pyqt5-dev-tools
! sudo apt-get install qttools5-dev-tools

# vmd_sizing の clone
! git clone  --depth 1 https://github.com/miu200521358/vmd_sizing.git
import time

elapsed_time = (time.time() - start_time) / 60

! echo "■■■■■■■■■■■■■■■■■■■■■■■■"
! echo "■　準備が終了しました"
! echo "■　"
! echo "■　処理にかかった時間：" "$elapsed_time" "分"
! echo "■■■■■■■■■■■■■■■■■■■■■■■■"


# VMDサイジング実行

変換パラメーターを入力して、セルを実行して下さい。

In [0]:
import os

#@markdown ### 調整対象vmdファイル
#@markdown 調整したいvmdファイルを、Googleドライブの「`vmdsizing`」フォルダにアップロードして、ファイル名を指定して下さい。
input_vmd = "test.vmd"  #@param {type: "string"}

#@markdown --- 

#@markdown ### トレース元モデルボーン構造CSVファイル
#@markdown トレース元モデルのボーン構造CSVファイルのパスを入力して下さい。
#@markdown ボーン構造CSVファイルを、Googleドライブの「`vmdsizing/born`」フォルダにアップロードして、ファイル名を指定して下さい。
born_model_csv = "\u3042\u306B\u307E\u3055\u5F0F\u30DF\u30AF.csv"  #@param {type: "string"}

#@markdown --- 

#@markdown ### トレース変換先モデルボーン構造CSVファイル
#@markdown トレース変換先モデルのボーン構造CSVファイルのパスを入力して下さい。
#@markdown ボーン構造CSVファイルを、Googleドライブの「`vmdsizing/born`」フォルダにアップロードして、ファイル名を指定して下さい。
#@markdown 頭部と腕の接触回避処理を行いたい場合、頂点構造CSVファイルをボーン構造CSVファイル名と同じ名前で、Googleドライブの「`vmdsizing/vertex`」フォルダにアップロードしてください。
replace_model_csv = "\u3042\u306B\u307E\u3055\u5F0F\u30DF\u30AF.csv"  #@param {type: "string"}



# 調整対象vmdファイルパス
input_vmd_path = base_path + "/"+ input_vmd
!echo 調整対象vmdファイル: "$input_vmd_path"

if not os.path.exists(input_vmd_path):
    print("■■■■■■■■■■■■■■■■■")
    print("■　**ERROR**　")
    print("■　調整対象vmdファイルが見つかりませんでした。")
    print("■■■■■■■■■■■■■■■■■")

# トレース元モデルボーン構造CSVファイルパス
born_model_csv_path = base_path + "/born/"+ born_model_csv
!echo トレース元モデルボーン構造CSVファイル: "$born_model_csv_path"

if not os.path.exists(born_model_csv_path):
    print("■■■■■■■■■■■■■■■■■")
    print("■　**ERROR**　")
    print("■　トレース元モデルボーン構造CSVファイルが見つかりませんでした。")
    print("■■■■■■■■■■■■■■■■■")

# トレース変換先モデルボーン構造CSVファイルパス
replace_model_csv_path = base_path + "/born/"+ replace_model_csv
!echo トレース変換先モデルボーン構造CSVファイル: "$replace_model_csv_path"

if not os.path.exists(replace_model_csv_path):
    print("■■■■■■■■■■■■■■■■■")
    print("■　**ERROR**　")
    print("■　トレース先モデルボーン構造CSVファイルが見つかりませんでした。")
    print("■■■■■■■■■■■■■■■■■")
    

# トレース変換先モデル頂点構造CSVファイルパス
replace_vertex_csv_path = base_path + "/vertex/"+ replace_model_csv
!echo トレース変換先モデル頂点構造CSVファイル: "$replace_vertex_csv_path"

if not os.path.exists(replace_model_csv_path):
    print("■■■■■■■■■■■■■■■■■")
    print("■　**ERROR**　")
    print("■　トレース先モデルボーン構造CSVファイルが見つかりませんでした。")
    print("■■■■■■■■■■■■■■■■■")

if not os.path.exists("./vmd_sizing"):
    print("■■■■■■■■■■■■■■■■■")
    print("■　**ERROR**　")
    print("■　VMDサイジング準備が実行されていません。")
    print("■■■■■■■■■■■■■■■■■")
    
if os.path.exists(input_vmd_path) and os.path.exists(born_model_csv_path) and os.path.exists(replace_model_csv_path) and os.path.exists("./vmd_sizing"):
    print("■■■■■■■■■■■■■■■■■")
    print("■　VMDサイジング処理実行")
    print("■■■■■■■■■■■■■■■■■")

    # とりあえず3ファイルあれば、処理実行
    ! cd ./vmd_sizing && python src/main.py --vmd_path "$input_vmd_path" --trace_bone_path "$born_model_csv_path" --replace_bone_path "$replace_model_csv_path" --replace_vertex_path "$replace_vertex_csv_path" --verbose 2

    print("■■■■■■■■■■■■■■■■■")
    print("■　VMDサイジング処理実行終了")
    print("■■■■■■■■■■■■■■■■■")
    
    !ls -l "$base_path"
else:
    print("■■■■■■■■■■■■■■■■■")
    print("■　**ERROR**　")
    print("■　準備に不備があるため、VMDサイジング処理は実行できませんでした。")
    print("■■■■■■■■■■■■■■■■■")
    
    

# !README

## エラーが起きた場合

### Googleドライブとの連携が行われていない

- 下記のようにメッセージが出た場合、Googleドライブとの連携が行われていない可能性があります。
- 【対応策】Googleドライブとの連携セクションのセルを実行してください。

------

![ドライブ連携失敗](https://drive.google.com/uc?export=view&id=17hkaOwJkR9IkFeQUfkiA3qc61DwVLHi1)

### 準備が出来ていない

- 下記のようにメッセージが出た場合、準備が行われていない可能性があります。
- 【対応策】準備実行セクションのセルを実行してください。

------

![準備未実行](https://drive.google.com/uc?export=view&id=1tjo0n4KovZQ9SfML08N6QAC46I0KeNFB)

### ファイルパスが正しくない

- 下記のようにメッセージが出た場合、Googleドライブのファイルパスが間違っている可能性があります。
- 【対応策】Googleドライブの「vmdsizing」フォルダ内のパスを確認してください。

------

![パラメーター不備](https://drive.google.com/uc?export=view&id=1MPdNbtc534THLxsla7LURPpUiSEWamyq)

### VMDモーションの読み込みに失敗している

- 下記のようにメッセージが出た場合、VMDモーションデータファイルの読み込みに失敗しています。
- 【対応策】VMDモーションデータをMMD上でモデルに適用させ、そのデータを「モーションデータ保存」で別名保存して、そのファイルを処理対象にしてください。
- 【注意点】モーションを適用させるMMDモデルはトレース元モデルか、もしくはトレース元モデルと同じボーン構造のモデルを使用してください。例えば、グルーブボーンがないモデルに読み込んでしまいますと、変換した後のデータからもグルーブボーンのモーションが消えてしまいます。

------

![デコード失敗](https://drive.google.com/uc?export=view&id=1p__jWtF7cDmrzzCEgtPBikx38m8Khsup)

 - 上記のどれにも当てはまらないエラーが発生した場合、ノートブックのコピーを私に共有してください。

## 任意事項

生成したモーションに関して、以下の行為は自由に行って下さい

 - モーションの調整・改変
 - ニコニコ動画、Youtube、Twitter等へのモーション使用動画投稿
   - 進捗等で生成したモーションそのままを投稿することも問題ありません。

**※不特定多数に対する配布は、必ず元々のモーションを作成された作者様にご確認ください。不用意に一般公開したり、配布したりなさらないよう、ご注意ください。**



## 禁止事項

生成したモーションに関して、以下の行為はご遠慮願います

 - モーションの完全自作発言
 - 各権利者様のご迷惑になるような行為
 - 営利目的の利用
 - 他者の誹謗中傷目的の利用（二次元・三次元不問）
 - 過度な暴力・猥褻・恋愛・猟奇的・政治的・宗教的表現を含む（R-15相当）作品への利用
 - その他、公序良俗に反する作品への利用


## 免責事項

 - 自己責任でご利用ください
 - ツール使用によって生じたいかなる問題に関して、作者は一切の責任を負いかねます

# ライセンス

Apache License 2.0

 - 変換したVMDモーションの結果を公開・配布する場合は、必ずライセンスの明記をお願い致します。
 - ニコニコ動画の場合、コンテンツツリーへ[配布動画(sm35044496)](https://www.nicovideo.jp/watch/sm35044496)を登録してください。

-----

```
ツール名：VMDサイジング
作者：miu200521358
```